## 1. 모듈/라이브러리 임포트

In [1]:
import torch, os
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable

## 2. 하이퍼 파라메터 세팅

In [3]:
max_epoch = 200
batch_size = 100
lr = 0.0002
z_dim = 64
output_dir = './output/'
if not os.path.exists(output_dir) : os.makedirs(output_dir)

## 3. 데이터 준비

In [4]:
# Image processing
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                     std=(0.5, 0.5, 0.5))])
# MNIST dataset
mnist = datasets.MNIST(root='./data/',
                       train=True,
                       transform=transform,
                       download=True)
# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

Processing...
Done!


## 4. 모델 정의

In [5]:
# Discriminator
D = nn.Sequential(
    nn.Linear(784, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 1),
    nn.Sigmoid())

# Generator
G = nn.Sequential(
    nn.Linear(z_dim, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 784),
    nn.Tanh())

if torch.cuda.is_available():
    D.cuda()
    G.cuda()

## 5. Loss/Optimizer

In [6]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(G.parameters(), lr=lr)

## 6. 기타 함수

In [7]:
# [-1,1] -> [0,1]
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

## 7. 트레이닝

In [8]:
# Start training
total_batch = len(data_loader.dataset)//batch_size
for epoch in range(max_epoch):
    for i, (images, _) in enumerate(data_loader):
        # Build mini-batch dataset
        images = Variable(images.view(batch_size, -1)).cuda()

        # Create the labels which are later used as input for the BCE loss
        real_labels = Variable(torch.ones(batch_size)).cuda()
        fake_labels = Variable(torch.zeros(batch_size)).cuda()

        #============= Train the discriminator =============#
        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs

        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = Variable(torch.randn(batch_size, z_dim)).cuda()
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs

        # Backprop + Optimize
        d_loss = d_loss_real + d_loss_fake
        D.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        #=============== Train the generator ===============#
        # Compute loss with fake images
        z = Variable(torch.randn(batch_size, z_dim)).cuda()
        fake_images = G(z)
        outputs = D(fake_images)

        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)

        # Backprop + Optimize
        D.zero_grad()
        G.zero_grad()
        g_loss.backward()
        g_optimizer.step()

        if (i+1) % 300 == 0:
            print('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, '
                  'g_loss: %.4f, D(x): %.2f, D(G(z)): %.2f'
                  %(epoch, max_epoch, i+1, total_batch, d_loss.data[0], g_loss.data[0],
                    real_score.data.mean(), fake_score.data.mean()))

    # Save real images
    if (epoch+1) == 1:
        images = images.view(images.size(0), 1, 28, 28)
        save_image(denorm(images.data), output_dir+'real_images.png')

    # Save sampled images
    fake_images = fake_images.view(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images.data), output_dir+'fake_images-%d.png' %(epoch+1))

# Save the trained parameters
torch.save(G.state_dict(), './generator.pkl')
torch.save(D.state_dict(), './discriminator.pkl')

/home/isjeon/anaconda3/envs/py36/lib/python3.6/site-packages/torch/nn/functional.py:1218: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch [0/200], Step[300/600], d_loss: 0.6070, g_loss: 2.8390, D(x): 0.89, D(G(z)): 0.36
Epoch [0/200], Step[600/600], d_loss: 0.2456, g_loss: 3.1135, D(x): 0.95, D(G(z)): 0.17
Epoch [1/200], Step[300/600], d_loss: 0.1118, g_loss: 4.4792, D(x): 0.94, D(G(z)): 0.04
Epoch [1/200], Step[600/600], d_loss: 0.1397, g_loss: 5.3275, D(x): 0.98, D(G(z)): 0.09
Epoch [2/200], Step[300/600], d_loss: 0.3665, g_loss: 3.7488, D(x): 0.87, D(G(z)): 0.13
Epoch [2/200], Step[600/600], d_loss: 0.3196, g_loss: 3.6628, D(x): 0.91, D(G(z)): 0.14
Epoch [3/200], Step[300/600], d_loss: 0.4247, g_loss: 3.3152, D(x): 0.94, D(G(z)): 0.25
Epoch [3/200], Step[600/600], d_loss: 0.8845, g_loss: 1.6951, D(x): 0.82, D(G(z)): 0.39
Epoch [4/200], Step[300/600], d_loss: 0.3990, g_loss: 3.3784, D(x): 0.89, D(G(z)): 0.18
Epoch [4/200], Step[600/600], d_loss: 0.7632, g_loss: 3.0822, D(x): 0.72, D(G(z)): 0.18
Epoch [5/200], Step[300/600], d_loss: 0.8606, g_loss: 2.5500, D(x): 0.74, D(G(z)): 0.19
Epoch [5/200], Step[600/600], d_

Epoch [46/200], Step[600/600], d_loss: 0.7333, g_loss: 2.5089, D(x): 0.81, D(G(z)): 0.29
Epoch [47/200], Step[300/600], d_loss: 0.7735, g_loss: 1.4042, D(x): 0.80, D(G(z)): 0.31
Epoch [47/200], Step[600/600], d_loss: 0.7537, g_loss: 2.1884, D(x): 0.74, D(G(z)): 0.24
Epoch [48/200], Step[300/600], d_loss: 1.0515, g_loss: 1.5660, D(x): 0.76, D(G(z)): 0.42
Epoch [48/200], Step[600/600], d_loss: 0.8534, g_loss: 1.5971, D(x): 0.64, D(G(z)): 0.17
Epoch [49/200], Step[300/600], d_loss: 0.8090, g_loss: 2.4734, D(x): 0.78, D(G(z)): 0.30
Epoch [49/200], Step[600/600], d_loss: 0.8165, g_loss: 1.8535, D(x): 0.69, D(G(z)): 0.23
Epoch [50/200], Step[300/600], d_loss: 0.6399, g_loss: 2.1278, D(x): 0.76, D(G(z)): 0.21
Epoch [50/200], Step[600/600], d_loss: 0.6397, g_loss: 2.0081, D(x): 0.83, D(G(z)): 0.28
Epoch [51/200], Step[300/600], d_loss: 0.7577, g_loss: 1.8194, D(x): 0.80, D(G(z)): 0.33
Epoch [51/200], Step[600/600], d_loss: 0.7750, g_loss: 1.8728, D(x): 0.69, D(G(z)): 0.21
Epoch [52/200], Step[

Epoch [93/200], Step[300/600], d_loss: 0.8911, g_loss: 1.7083, D(x): 0.66, D(G(z)): 0.23
Epoch [93/200], Step[600/600], d_loss: 0.9002, g_loss: 1.4231, D(x): 0.75, D(G(z)): 0.32
Epoch [94/200], Step[300/600], d_loss: 0.7434, g_loss: 1.8537, D(x): 0.73, D(G(z)): 0.23
Epoch [94/200], Step[600/600], d_loss: 0.8026, g_loss: 1.7477, D(x): 0.71, D(G(z)): 0.24
Epoch [95/200], Step[300/600], d_loss: 0.9176, g_loss: 1.0610, D(x): 0.78, D(G(z)): 0.38
Epoch [95/200], Step[600/600], d_loss: 0.8142, g_loss: 1.8156, D(x): 0.74, D(G(z)): 0.26
Epoch [96/200], Step[300/600], d_loss: 0.7935, g_loss: 1.4520, D(x): 0.82, D(G(z)): 0.35
Epoch [96/200], Step[600/600], d_loss: 0.9283, g_loss: 1.4922, D(x): 0.73, D(G(z)): 0.33
Epoch [97/200], Step[300/600], d_loss: 0.9452, g_loss: 1.7396, D(x): 0.67, D(G(z)): 0.29
Epoch [97/200], Step[600/600], d_loss: 0.8126, g_loss: 1.6947, D(x): 0.70, D(G(z)): 0.25
Epoch [98/200], Step[300/600], d_loss: 0.8017, g_loss: 1.5833, D(x): 0.74, D(G(z)): 0.27
Epoch [98/200], Step[

Epoch [139/200], Step[300/600], d_loss: 0.7450, g_loss: 1.6266, D(x): 0.76, D(G(z)): 0.27
Epoch [139/200], Step[600/600], d_loss: 0.8278, g_loss: 1.8595, D(x): 0.72, D(G(z)): 0.27
Epoch [140/200], Step[300/600], d_loss: 0.9380, g_loss: 1.7361, D(x): 0.74, D(G(z)): 0.34
Epoch [140/200], Step[600/600], d_loss: 0.7453, g_loss: 1.6876, D(x): 0.73, D(G(z)): 0.24
Epoch [141/200], Step[300/600], d_loss: 0.7705, g_loss: 1.5515, D(x): 0.77, D(G(z)): 0.31
Epoch [141/200], Step[600/600], d_loss: 0.8757, g_loss: 1.7127, D(x): 0.66, D(G(z)): 0.22
Epoch [142/200], Step[300/600], d_loss: 0.8582, g_loss: 1.7548, D(x): 0.76, D(G(z)): 0.34
Epoch [142/200], Step[600/600], d_loss: 0.9082, g_loss: 1.8117, D(x): 0.77, D(G(z)): 0.36
Epoch [143/200], Step[300/600], d_loss: 0.9538, g_loss: 1.8761, D(x): 0.61, D(G(z)): 0.21
Epoch [143/200], Step[600/600], d_loss: 0.9108, g_loss: 1.4573, D(x): 0.77, D(G(z)): 0.38
Epoch [144/200], Step[300/600], d_loss: 0.9422, g_loss: 1.6263, D(x): 0.70, D(G(z)): 0.33
Epoch [144

Epoch [185/200], Step[300/600], d_loss: 0.8334, g_loss: 1.8844, D(x): 0.75, D(G(z)): 0.33
Epoch [185/200], Step[600/600], d_loss: 0.8208, g_loss: 1.9035, D(x): 0.70, D(G(z)): 0.27
Epoch [186/200], Step[300/600], d_loss: 0.7365, g_loss: 2.1242, D(x): 0.71, D(G(z)): 0.20
Epoch [186/200], Step[600/600], d_loss: 0.8370, g_loss: 2.2128, D(x): 0.70, D(G(z)): 0.26
Epoch [187/200], Step[300/600], d_loss: 0.8632, g_loss: 1.8579, D(x): 0.72, D(G(z)): 0.28
Epoch [187/200], Step[600/600], d_loss: 0.8340, g_loss: 1.6581, D(x): 0.71, D(G(z)): 0.27
Epoch [188/200], Step[300/600], d_loss: 0.9783, g_loss: 1.4394, D(x): 0.73, D(G(z)): 0.34
Epoch [188/200], Step[600/600], d_loss: 0.8547, g_loss: 1.7786, D(x): 0.70, D(G(z)): 0.24
Epoch [189/200], Step[300/600], d_loss: 0.7160, g_loss: 1.7267, D(x): 0.73, D(G(z)): 0.23
Epoch [189/200], Step[600/600], d_loss: 0.8522, g_loss: 1.4764, D(x): 0.83, D(G(z)): 0.37
Epoch [190/200], Step[300/600], d_loss: 0.8223, g_loss: 1.7958, D(x): 0.67, D(G(z)): 0.20
Epoch [190